# Quickstart Tutorial: Neurodesktop & BIDS Conversion

Welcome! This tutorial will guide you through:
- Launching Neurodesktop tools
- Converting XNAT DICOM data to BIDS format
- Running common neuroimaging pipelines (MRIQC, fMRIPrep)

In Jupyter click **View - Table of Contents** to open the list of available sections.

## Loading necessary packages for tutorial

In order to run
## Table of Contents
- [1️⃣ Intro](#Intro)
- [2️⃣ Data Conversion](#Conversion)
  - [2.1 Installing xnat_dcm2bids](#Install)
  - [2.2 Important Tools & Links](#Tools)
  - [2.3 XNAT Connection](#XNAT-Connection)
  - [2.4 Downloading Session List](#Download-Sessions)
  - [2.5 Preparing BIDS Directory](#Prepare-BIDS)
  - [2.6 Config File Setup](#Config)
  - [2.7 Converting Sessions](#Convert-Sessions)
- [3️⃣ Software & Pipelines](#Software)
  - [3.1 MRIQC](#MRIQC)
  - [3.2 fMRIPrep](#fMRIPrep)
  - [3.3 MATLAB & FSL](#Preinstalled)
  - [3.4 Other Programs](#Other-Programs)
  

## 1️⃣ Intro <a id='Intro'></a>

This tutorial will guide you through launching Neurodesktop tools using Jupyter Notebook, however you can also run those tools in Terminal. Just use the commands below without exclamation mark "❗"


Start by launching Neurodesktop and choosing one of the following:
- **Terminal**
- **Jupyter Notebook**
- **Remote Desktop (Neurodesktop GUI)**

In the Neurodesktop GUI you can run MATLAB, FSL, Freeview, and other programs directly. See [Software](#Software) for details.

## 2️⃣ Data Conversion <a id='Conversion'></a>

### 2.1 Install `xnat_dcm2bids` <a id='Install'></a>

You can install this package on MacOS, Windows, or Linux:


In [ ]:
!pip install --force-reinstall https://github.com/nencki-lobi/xnat_dcm2bids/archive/refs/heads/master.zip > /dev/null 2>&1

### 2.2 Important Tools & Links <a id='Tools'></a>
   - **xnat_dcm2bids** - downloads data from a selected XNAT session (e.g., `f490f566-2124-46`) and converts it to BIDS format using `dcm2bids`:  
`xnat_dcm2bids --bids-dir ~/bids-dir --config ./code/config.json f490f566-2124-46 03 01` 
   - **dcm2bids_scaffold** - prepares a new BIDS directory  
     `dcm2bids_scaffold -o /home/jovyan/bids-dir`
   - **xnat-get** - downloads all DICOM files from XNAT  
     `xnat-get -p mj_ris -t ./sourcedata`
   - **xnat_getcsv** - downloads session list to CSV  
     `xnat_getcsv mj_ris.csv mj_ris`
   - **lobi_script** - run scripts from [lobi-mri-scripts](https://github.com/nencki-lobi/lobi-mri-scripts)  
     `lobi_script run_mriqc.sh 03 ~/bids-dir ~/bids-dir/derivatives/mriqc`

Find out more in [xnat_dcm2bids repository](https://github.com/nencki-lobi/xnat_dcm2bids)

### 2.3 Configure XNAT Connection <a id='XNAT-Connection'></a>

xnat_dcm2bids tools use specific authentication method. It is neccesarry to initialize the connection for the first time in Terminal - type `xnat-get` and the provide the url "xnat.nencki.edu.pl" and Your credentials.
Expected output ✅:
```
ERROR! project_id ("-p") must be provided to use empty IDs string
```

### 2.4 Download Session List <a id='Download-Sessions'></a>

In [ ]:
%cd ~
!xnat_getcsv mj_ris.csv mj_ris

/home/jovyan
🟢 Data saved to mj_ris.csv


In [ ]:
#display content of "mj_ris.csv"
import pandas as pd
df = pd.read_csv("mj_ris.csv", dtype=str, index_col=0)
df.head(10)

### 2.5 Prepare BIDS Directory <a id='Prepare-BIDS'></a>

In [ ]:
bidsdir="/home/jovyan/bids-dir"

In [ ]:
!dcm2bids_scaffold -o {bidsdir}

In [ ]:
%cd {bidsdir}

/home/jovyan/bids-dir


In [ ]:
!ls -1 {bidsdir}

CHANGES
README
code
dataset_description.json
derivatives
participants.json
participants.tsv
sourcedata
sub-03
tmp_dcm2bids


### 2.6 Setup Config File <a id='Config'></a>

You can either prepare config file on Your own or ask us to provide one. Some examples are available in the [lobi-mri-scripts repository](https://github.com/nencki-lobi/lobi-mri-scripts/tree/main/dcm2bids). If You have already cloned the repository, then You can link one of the configuration files as follows.

In [ ]:
!ln -sf ~/lobi-mri-scripts/dcm2bids/mjris.json {bidsdir}/code/config.json

### 2.7 Convert Sessions <a id='Convert-Sessions'></a>

**Single session example:**
- XNAT session ID: `f490f566-2124-46`
- Subject: `03`
- Session: `01`


In [ ]:
!xnat_dcm2bids --bids-dir {bidsdir} --config ./code/config.json f490f566-2124-46 03 01

**Multiple sessions from CSV:**


In [ ]:
!while IFS=',' read -r id _ sub ses _; do \
  echo xnat-dcm2bids --config ./code/config.json "$id" "$sub" 1; \
done < ~/mj_ris.csv

## 3️⃣ Software & Pipelines <a id='Software'></a>

### 3.1 MRIQC <a id='MRIQC'></a>

Run MRIQC for a single subject:


In [ ]:
!lobi_script run_mriqc.sh 03 {bidsdir} {bidsdir}/derivatives/mriqc

### 3.2 fMRIPrep <a id='fMRIPrep'></a>

Run fMRIPrep for multiple subjects from CSV:


In [ ]:
!tail -n +2 ~/mj_ris.csv | parallel --colsep ',' echo lobi_script run_fmriprep_min.sh {{3}} {bidsdir} {bidsdir}/derivatives/fmriprep

### 3.3 Preinstalled software <a id='Preinstalled'></a>

Graphical tools do not work efficiently via `module load` from Neurodesktop repository. Instead, follow these steps to make it work from local:
- **MATLAB**: run in Terminal with `./matlab`
- **FSL**: configure first with `source ~/shared_storage/fsl.sh`, then run eg. `fsleyes`

### 3.4 Other programs 

Other programs including graphical viewers such as Freeview, mrtrix etc. are not available locally and should be run from Neurodesktop via `module load` or `ml` eg. `ml ants` and in Jupyter as follows:

In [2]:
import module
await module.load('ants')

In [3]:
!antsRegistration


COMMAND: 
     antsRegistration
          This program is a user-level registration application meant to utilize classes 
          in ITK v4.0 and later. The user can specify any number of "stages" where a stage 
          consists of a transform; an image metric; and iterations, shrink factors, and 
          smoothing sigmas for each level. Note that explicitly setting the 
          dimensionality, metric, transform, output, convergence, shrink-factors, and 
          smoothing-sigmas parameters is mandatory. 

OPTIONS: 
     --version 
          Get Version Information. 

     -d, --dimensionality 2/3/4
          This option forces the image to be treated as a specified-dimensional image. If 
          not specified, we try to infer the dimensionality from the input image. 

     -o, --output outputTransformPrefix
                  [outputTransformPrefix,<outputWarpedImage>,<outputInverseWarpedImage>]
          Specify the output transform prefix (output format is .nii.gz ). Opti